# Projet: Débordement d'égouts
## Rapport équipe 9 - A19
#### Noboru Yoshida - Mehdi Chaid - Mathieu Giroud-Huppé - Maxime Gosselin
#### 05 Décembre 2020

## Table des matières

1. [Objectif](#objectives)  
2. [Méthode choisie](#chosen_method)
3. [Traitement des données](#data_processing)  
    1. [Chargement des données de l'ensemble d'entraînement](#data_loading) 
    2. [Nettoyage des données](#data_cleaning)  
    3. [Augmentation des données](#data_augmentation)
4. [Analyse exploratoire](#data_analysis)  
    1. [Distance à la station météorologique](#distance-meteo) 
    2. [Hauteur du trop plein](#very-full-height)
    3. [TODO: Continuer](#continue)
5. [Sélection de modèles](#model-selection)
    1. [Régression logistique](#logistic-reg)
    2. [Classification bayésienne naive](#naive-bayes)
    3. [Forêt aléatoire](#random-forest)
    4. [Séparateur à vaste marge](#svm)
6. [Améliorations possibles](#possible-improvements)

<a id="objectives"></a>

## 1. Objectif

Ce rapport vise à fournir une prédiction sur les surverses dans plusieurs ouvrages sur l'île de Montréal. TODO: Continuer

<a id="chosen_method"></a>

## 2. Méthode choisie

<a id="data_processing"></a>

## 3. Traitement des données

TODO

In [ ]:
using CSV, DataFrames, GLM, Statistics, Dates, Gadfly, Random, MLBase;
include("utils/precipitation.jl");

### 3.1. Ouvrages_surverse.csv

Ici nous chargeons les données provenant du fichier <i>ouvrages-surverses.csv</i>. Il y a plusieurs colonnes dans cette table, mais nous sommes intéréssé que par 4 de ces colonnes. ID_ouvrage, Latitude, Longitude, TP_Z.

In [ ]:
ouvrages = CSV.read("data/ouvrages-surverses.csv");
colnames = ["N_Env", "ID_SOMA", "ID_OUVRAGE", "NOM", "SOMA_SEC", "REGION", "TP_X", "TP_Y", "TP_Z", "TP_LAT", "TP_LNG", "EMI_X", "EMI_Y", "EMI_LNG", "EMI_LAT"];
names!(ouvrages, Symbol.(colnames));
select!(ouvrages, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z]);

#### Inputation des hauteurs de Trop-plein manquant par la moyenne des hauteurs des autres trop plein.

In [ ]:
ouvrages.TP_Z = coalesce.(ouvrages.TP_Z, mean(ouvrages[completecases(ouvrages), :].TP_Z));
first(shuffleDf(ouvrages), 5)

#### Visualtion des données chargées

In [ ]:
plot(ouvrages, x=:TP_Z, Geom.histogram(bincount=50), Guide.xlabel("Height of TropPlein"),Guide.ylabel("Frequency"))

In [ ]:
plot(ouvrages,x=:TP_LNG, y=:TP_LAT, Geom.point)

### 3.2. Surverses.csv

Ici nous chargeons les données du fichier <i>Surverses.csv</i>. Nous utilisons toutes les colonnes de ce taleau. Le fichier est structuré comme ceci : <br>

|NO_Ouvrage|Date|Surverse|Raison|
|---|---|---|---|
|String|Date|Bool|String|

NO_ouvrage : <br>
Date : <br>
Surverse : <br>
Raison : <br>

Au final, nous gardons que les trois premières colonnes. La colonne NO_Ouvrage sera renommé afin de servir de clé externe pour se joindre à la table des ouvrages créés plus haut.

In [ ]:
surverses = CSV.read("data/surverses.csv", missingstring="-99999");
first(shuffleDf(surverses),5)

#### Garder les mois qui nous concerne. (Mai à Octobre)

In [ ]:
surverses = filter(row -> month(row.DATE) > 4, surverses);
surverses = filter(row -> month(row.DATE) < 11, surverses);

#### Garder les raisons de surverse qui nous intéresse. (Pluie, Inconnue, TS)

In [ ]:
raison = coalesce.(surverses[:,:RAISON],"Inconnue");
surverses[!,:RAISON] = raison;

surverses = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], surverses);
select!(surverses, [:NO_OUVRAGE, :DATE, :SURVERSE]);

#### Retirer les données manquantes ainsi que renommer la colonne NO_OUVRAGE en ID_OUVRAGE

In [ ]:
surverses = dropmissing(surverses, disallowmissing=true);
rename!(surverses, :NO_OUVRAGE => :ID_OUVRAGE);
first(shuffleDf(surverses), 5)

### 3.3 Precipitation.csv

Ici nous chargeons les données du fichier <i> Precipitation.csv </i>. Nous devons effectuer des fonctions d'aggrégation sur ce tableau avant de pouvoir l'intégré dans notre jeu de donnée de nos modèles. En effet, ce tableau contient les données de précipitation par heure pour chacune des cinqs stations météorologiques près de montréal. Cependant, notre jeu de données des features est seulement par jour. Nous allons donc applique 3 fonctions d'aggrégation différente, soit : Somme, Maximum et Maximum3h. 


Date, Heure, McTavish, Bellevue, Assomption, Trudeau, StHubert

Contient toutes les données de précipitation des années 2013 à 2019 (train et test)

In [ ]:
precipitations = CSV.read("data/precipitations.csv",missingstring="-99999");
rename!(precipitations, Symbol("St-Hubert")=>:StHubert);

#### Garder les mois qui nous concerne. (Mai à Octobre)

In [ ]:
precipitations = filter(row -> month(row.date) > 4, precipitations);
precipitations = filter(row -> month(row.date) < 11, precipitations);
first(shuffleDf(precipitations),5)

#### Inputation des données de précipitation manquante par la moyenne de la journée de la station concernée

In [ ]:
precipitation_by_day = by(precipitations, :date,  
                            McTavish = :McTavish=>mean_wo_missing, 
                            Bellevue = :Bellevue=>mean_wo_missing, 
                            Assomption = :Assomption=>mean_wo_missing,
                            Trudeau = :Trudeau=>mean_wo_missing,
                            StHubert = :StHubert=>mean_wo_missing)

for i=1:size(precipitations,1)
    if isequal(precipitations[i, :McTavish], missing)
        precipitations[i,:McTavish] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:McTavish][1]
    end
    if isequal(precipitations[i, :Bellevue], missing)
        precipitations[i,:Bellevue] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:Bellevue][1]
    end
    if isequal(precipitations[i, :Assomption], missing)
        precipitations[i,:Assomption] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:Assomption][1]
    end
    if isequal(precipitations[i, :Trudeau], missing)
        precipitations[i,:Trudeau] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:Trudeau][1]
    end
    if isequal(precipitations[i, :StHubert], missing)
        precipitations[i,:StHubert] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:StHubert][1]
    end
end

first(shuffleDf(precipitations), 5)

####  Aggrégation des précipitations

##### Somme des précipitations par jour par station météo

In [ ]:
pcp_sum = by(precipitations, :date,  
            McTavish = :McTavish=>sum, 
            Bellevue = :Bellevue=>sum,
            Assomption = :Assomption=>sum, 
            Trudeau = :Trudeau=>sum, 
            StHubert = :StHubert=>sum);
first(shuffleDf(pcp_sum), 5)

##### Visualisation des précipitations par station météo

In [ ]:
df_for_plot = pcp_sum
df_for_plot = filter(row -> year(row.date) == 2016, pcp_sum);
df_for_plot = melt(df_for_plot, :date)

plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

##### Maximum des précipitations par jour par station météo

In [ ]:
pcp_max = by(precipitations, :date,  
            McTavish = :McTavish=>maximum,
            Bellevue = :Bellevue=>maximum, 
            Assomption = :Assomption=>maximum,
            Trudeau = :Trudeau=>maximum,
            StHubert = :StHubert=>maximum)
first(shuffleDf(pcp_max),5)

##### Visualisation des précipitations par station météo

In [ ]:
df_for_plot = pcp_max
df_for_plot = filter(row -> year(row.date) == 2013, pcp_max);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

##### Maximum de précipitation durant 3 heures consécutives par heure par station météo

In [ ]:
pcp_max3h = by(precipitations, :date,
                McTavish = :McTavish=>maximum3,
                Bellevue = :Bellevue=>maximum3,
                Assomption = :Assomption=>maximum3,
                Trudeau = :Trudeau=>maximum3,
                StHubert = :StHubert=>maximum3)
first(shuffleDf(pcp_max3h),5)

##### Visualisation des précipitations par station météo

In [ ]:
df_for_plot = pcp_max3h
df_for_plot = filter(row -> year(row.date) == 2018, pcp_sum);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

#### Visualisation des précipitation aggégrés d'une station pour une année

In [ ]:
date_to_pt = 2018;
loc = :McTavish
mct_sum = pcp_sum[:,[1,2]]
rename!(mct_sum, loc => :Sum);
mct_sum = filter(row -> year(row.date) == date_to_pt, mct_sum);

mct_max = pcp_max[:,[1,2]]
rename!(mct_max,loc => :Max);
mct_max = filter(row -> year(row.date) == date_to_pt, mct_max);

mct_max3h = pcp_max3h[:,[1,2]]
rename!(mct_max3h,loc => :Max3h);
mct_max3h = filter(row -> year(row.date) == date_to_pt, mct_max3h);

df_for_plot = join(mct_sum, mct_max3h, on = :date);
df_for_plot = join(df_for_plot, mct_max, on = :date);
df_for_plot = melt(df_for_plot, :date)


plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

### Jointure des Dataframes

#### Chargement de l'ensemble de test et jointure de ouvrages et surverses

In [ ]:
X_test = CSV.read("data/test.csv"); #NO_OUVRAGE, DATE
rename!(X_test, :NO_OUVRAGE => :ID_OUVRAGE);

X_test = join(ouvrages, X_test, on =:ID_OUVRAGE);
features = join(ouvrages, surverses, on =:ID_OUVRAGE);

srv = filter(row -> row.SURVERSE == 1, features);
first(shuffleDf(srv), 10)

#### Visualisation des stations en surverse et en non-surverse à une date donnée

In [ ]:
# df_for_geo_plot = filter(row -> row.DATE == Date(2018,7,25), features)
df_for_geo_plot = filter(row -> row.DATE == Date(2015, 5, 30), features)
# df_for_geo_plot = filter(row -> row.DATE == Date(2017, 10, 9), features)
df_for_geo_plot[:SURVERSE] = convert(Array{Bool,1}, df_for_geo_plot[:SURVERSE])
plot(df_for_geo_plot, x=:TP_LNG, y=:TP_LAT, Geom.point, color=:SURVERSE, Guide.title("2018-07-25, état des surverses"))
#first(df_for_geo_plot,5) 

#### Préparation afin d'assigner les données de précipitations aux ouvrages

In [ ]:
station_df = DataFrame(STATION = String[], LAT = Float64[], LNG = Float64[]);

push!(station_df, ["McTavish",   45.504742, -73.579167]);
push!(station_df, ["Bellevue",   45.427222, -73.929167]);
push!(station_df, ["Assomption", 45.809444, -73.434722]);
push!(station_df, ["Trudeau",    45.467778, -73.741667]);
push!(station_df, ["StHubert",   45.517500, -73.416944]);

station_df

#### Ajout de colonnes vides dans les jeux de données

In [ ]:
function addColsForPrecipationPerDay(df)
    df[!, :PCP_SUM] = zeros(size(df, 1));
    df[!, :PCP_MAX] = zeros(size(df, 1));
    df[!, :PCP_MAX3] = zeros(size(df, 1));
    df[!, :METEO] = fill("", size(df, 1));
    return df
end

X_test = addColsForPrecipationPerDay(X_test)
permutecols!(X_test, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :DATE, :METEO, :PCP_SUM, :PCP_MAX, :PCP_MAX3]);
    
features = addColsForPrecipationPerDay(features)
permutecols!(features, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :DATE, :METEO, :PCP_SUM, :PCP_MAX, :PCP_MAX3, :SURVERSE]);

first(shuffleDf(features), 5)

#### Jointure des précipitations aggrégés aux jeux de données basé sur la station météo la plus près de l'ouvrage concerné

In [ ]:
function fillPrecipitationWithClosestMeteoStation(df)
    for i=1:size(df, 1)
        id_ouvrage = df[i, 1]; 
        closest_station = "McTavish"; # initial value
        shortest_dist = -1;

        # Find closest station
        for j=1:size(station_df, 1)
            dist = findDistance(df[i, :TP_LAT], df[i, :TP_LNG], station_df[j, :LAT], station_df[j, :LNG]);

            if shortest_dist == -1 || dist < shortest_dist
                shortest_dist = dist;
                closest_station = station_df[j, :STATION];
            end
        end

        # Augment comb with a weighted p_sum, based on the distance to the station
        p_sum = pcp_sum[∈([df[i, :DATE]]).(pcp_sum.date), Symbol(closest_station)];
    #     comb[i, :PCP_SUM] = p_sum[1] * (1 - shortest_dist);
        df[i, :PCP_SUM] = p_sum[1]; 

        # Augment comb with a weighted p_max, based on the distance to the station
        p_max = pcp_max[∈([df[i, :DATE]]).(pcp_max.date), Symbol(closest_station)]
    #     comb[i, :PCP_MAX] = p_max[1] * (1 - shortest_dist);
        df[i, :PCP_MAX] = p_max[1];

        # Augment comb with a weighted p_max3h, based on the distance to the station
        p_max3 = pcp_max3h[∈([df[i, :DATE]]).(pcp_max3h.date), Symbol(closest_station)]
    #     comb[i, :PCP_MAX3] = p_max3[1] * (1 - shortest_dist);
        df[i, :PCP_MAX3] = p_max3[1]; 

        df[i, :METEO] = closest_station
    end
    return df
end

In [ ]:
X_test = fillPrecipitationWithClosestMeteoStation(X_test)
features = fillPrecipitationWithClosestMeteoStation(features)
first(shuffleDf(features), 5)

#### Retrait des données extrêmes de précipitation. Purement subjectif.

In [ ]:
X_test[X_test[:PCP_SUM] .> 750, :PCP_SUM] = 750;
X_test[X_test[:PCP_MAX] .> 500, :PCP_MAX] = 500;
X_test[X_test[:PCP_MAX3] .> 750, :PCP_MAX3] = 750;

features[features[:PCP_SUM] .> 750, :PCP_SUM] = 750;
features[features[:PCP_MAX] .> 500, :PCP_MAX] = 500;
features[features[:PCP_MAX3] .> 750, :PCP_MAX3] = 750;

#### Visualisation des stations et leur station météo associée

In [ ]:
df_for_geo_plot = filter(row -> row.DATE == Date(2018,7,25), features)
df_for_geo_plot[:SURVERSE] = convert(Array{Bool,1}, df_for_geo_plot[:SURVERSE])
plot(df_for_geo_plot, x=:TP_LNG, y=:TP_LAT, Geom.point, color=:METEO, Guide.title("2018-07-25, Regroupement par station météo"))
#first(df_for_geo_plot,5) 

#### Ouvrage 3350-07D pour 2018, quand il surverse, basé sur le critère max3h de pluie tombé.

In [ ]:

# 3260-01D dans Rivière-des-Prairies
# 3350-07D dans Ahunstic
# 4240-01D dans Pointe-aux-Trembles
# 4350-01D dans le Vieux-Montréal
# 4380-01D dans Verdun

id_ouvrage_to_show = "3350-07D"
df_temp = filter(row -> row.ID_OUVRAGE ∈ [id_ouvrage_to_show], features)
df_temp = filter(row -> year(row.DATE) == 2018, df_temp);
df_temp = df_temp[!,[:ID_OUVRAGE, :DATE, :PCP_MAX3, :SURVERSE]]
df_temp[:SURVERSE] = convert(Array{Bool,1}, df_temp[:SURVERSE])



plot(df_temp, x=:DATE, y=:PCP_MAX3, Geom.point, color=:SURVERSE,Guide.title(id_ouvrage_to_show))
#first(shuffleDf(df_temp), 10)

#### Ajout de colonnes Mois (MONTH) et Jour (DAY)

In [ ]:
function splitDateByMonthAndDay(df)
    df[!,:MONTH] = month.(df.DATE);
    df[!,:DAY] = day.(df.DATE);
    return df
end

X_test = splitDateByMonthAndDay(X_test)
features = splitDateByMonthAndDay(features)
first(shuffleDf(features[!, [:DATE, :MONTH, :DAY]]), 5)

#### Déclaration de fonctions utilitaires

In [ ]:
function partitionTrainTest(data, at = 0.8) # https://discourse.julialang.org/t/simple-tool-for-train-test-split/473/2
    n = nrow(data)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    return data[train_idx,:], data[test_idx,:]
end


In [ ]:
function standarizeTrainTestCol(X_train, X_test, col)
    mean_train_col = mean(X_train[!, col]);
    std_train_col = std(X_train[!, col]);
    X_train[!, col] = (X_train[!, col] .- mean_train_col) ./ std_train_col;
    X_test[!, col] = (X_test[!, col] .- mean_train_col) ./ std_train_col;
    
    return X_train, X_test
end
    

In [ ]:
function oneHotTrainTestCol(X_train, X_test, col, nb_cat)
    
end

### Standardization des colonnes et création de l'ensemble de validation

In [ ]:
X_train = features

X_train, X_test = standarizeTrainTestCol(X_train, X_test, :TP_LAT)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :TP_LNG)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :TP_Z)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :PCP_SUM)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :PCP_MAX)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :PCP_MAX3)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :MONTH)
X_train_all, X_test = standarizeTrainTestCol(X_train, X_test, :DAY)

X_train, X_val = partitionTrainTest(X_train_all, 0.8)

first(shuffleDf(X_train),5)

In [ ]:
first(shuffleDf(X_test),5)

In [ ]:
first(shuffleDf(X_val),5)

## Chargement des données

<a id="data_analysis"></a>

## 4. Analyse exploratoire


